In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import urllib
import time

In [2]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [3]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):

    try:
        # build the flipkart query
        search_url = "https://www.google.com/search?tbm=isch&q={0}"

        image_urls = set()
        image_count = 0
        results_start = 0
        google_cookies_accepted = False
        search_url = search_url.format(query).replace (" ","%20")
        page_source = wd.page_source

        # Accept googlecookies button if not already accepted
        if not google_cookies_accepted:
            wd = accept_google_cookies(wd, search_url)

        while image_count < max_links_to_fetch-1:
            scroll_to_end(wd)

            #Click thumbnails ang get main image of better resolution
            for thumbnail in wd.find_elements(By.CLASS_NAME, "rg_i"):
                thumbnail.click();
                search_page_html = bs(wd.page_source)
                thumbnail_results = search_page_html.findAll("img", {"class": "n3VNCb"})
                image_urls = image_urls.union(extract_image_urls(thumbnail_results))
                if len(image_urls) >= max_links_to_fetch:
                    break
                    
            image_count = len(image_urls)
            # Extract more images to meet the count
            if image_count >= max_links_to_fetch:
                print(f"Found: {image_count} image links, done!")
                break
            else:
                print("Found:", image_count, "image links, looking for more ...")

                # Check if more search result pages available
                load_more_button = wd.find_element(By.CLASS_NAME, "mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
    except Exception as e:
        print(f"ERROR while fetching image URLs - {e}")

    return image_urls

In [4]:
def accept_google_cookies(wd: webdriver, url: str):
    try:
        wd.get(url)
        # Find if there is accept all button, if so click it
        wd.find_element(By.XPATH, "//*[@id='yDmH0d']/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button").click();
    except Exception as e:
        print(f"ERROR while accepting the google cookies: - {url} - {e}")
        
    return wd    

In [5]:
def extract_image_urls(thumbnail_results: list):
    image_urls = set()
    try:
        for img in thumbnail_results:
            # extract image urls
            if img['src'] and 'http' in img['src']:
                image_urls.add(img['src'])
                
    except Exception as e:
        print(f"ERROR - Extracting the image URLs - {e}")
    return image_urls

In [6]:
def build_beautifulsoup_object(url: str):
    try:
        # get all image thumbnail results
        uClient = uReq(url)
        flipkartPage = uClient.read()
        uClient.close()
        search_page_html = bs(flipkartPage, "html.parser")
        
    except Exception as e:
        print(f"ERROR - building the Beautiful Soup object {url} - {e}")
        
    return search_page_html                               

In [7]:
def download_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [8]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    try:
        target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

        if not os.path.exists(target_folder):
            os.makedirs(target_folder)

        with webdriver.Chrome(executable_path=driver_path) as wd:
            res = fetch_image_urls(search_term, number_images, wd, sleep_between_interactions=0.5)

        counter = 0
        for elem in res:
            download_image(target_folder, elem, counter)
            counter += 1
    except Exception as e:
        print(f"ERROR - while search and download images - {e}")

In [9]:
DRIVER_PATH = r'./chromedriver.exe'
search_term = 'Narendra Damodar Das Modi'
# num of images you can pass it from here  by default it's 10 if you are not passing
#number_images = 50
search_and_download(search_term=search_term, driver_path=DRIVER_PATH, number_images=50)

C:\Users\ashwin.prabhu\AppData\Local\Temp\ipykernel_23576\3963979008.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:


Found: 50 image links, done!
SUCCESS - saved https://shabdabramha.files.wordpress.com/2014/04/jashodaben-narendra-modi.jpg - as ./images\narendra_damodar_das_modi
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQg70hMkXdCL5C5Cwvtd2TVe2jbukOekwCK5g&usqp=CAU - as ./images\narendra_damodar_das_modi
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcReo-l-FRmnPJchpOFfS0wnAZ9eRGtm5vBVFw&usqp=CAU - as ./images\narendra_damodar_das_modi
SUCCESS - saved https://i0.wp.com/www.eurasiareview.com/wp-content/uploads/2015/08/Modi.jpg?resize=800%2C445&ssl=1 - as ./images\narendra_damodar_das_modi
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSmvccxCLuohKyXJAL_i7_AwbSE2ZNGSJAoKg&usqp=CAU - as ./images\narendra_damodar_das_modi
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT9eqQiBl10_mDp1CB-9igoQAn_P4JiMYO_rQ&usqp=CAU - as ./images\narendra_damodar_das_modi
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?